In [ ]:
import sys
sys.path.append('..')

In [ ]:
from binance.spot import Spot
from secret import API_KEY, SECRET_KEY
from crypto.utils import *
import pandas as pd


client = Spot()
client_time = client.time()

client = Spot(
    key=API_KEY,
    secret=SECRET_KEY)

# Get account information
# print(client.account())


In [ ]:
from dateutil.rrule import rrule, MONTHLY
from datetime import datetime

def month_iter(start_month, start_year, end_month, end_year):
    start = datetime(start_year, start_month, 1)
    end = datetime(end_year, end_month, 1)

    return ((d.month, d.year) for d in rrule(MONTHLY, dtstart=start, until=end))

    

In [ ]:
loan_history = []
for m in month_iter(1, 2021, 4, 2022):
    start_date = "{0}/{1}/01"
    start = date_to_milliseconds(start_date.format(m[1], m[0]))
    if m[0]==12:
        end = date_to_milliseconds(start_date.format(m[1]+1, 1)) - 1
    else:
        end = date_to_milliseconds(start_date.format(m[1], m[0]+1)) - 1

    s = milliseconds_to_date(start)
    e = milliseconds_to_date(end)
    print(f'Getting data for: {s} to {e}')
    loan_history.extend(client.loan_history('BUSD', startTime=start, endTime=end))

In [ ]:
df = pd.DataFrame(loan_history)
df.amount = df.amount.astype(str).astype(float)
df.timestamp = df.timestamp.apply(milliseconds_to_date)

In [ ]:
df.type.unique()

In [ ]:
df[df.type=="borrowIn"].amount.sum()

In [ ]:
df = df.rename(columns={
    "timestamp": "Date(UTC)",
    "asset": "Coin",
    "amount": "Amount",
    "tranId": "PaymentID"
}
)

df["Indicated Amount"] = df["Amount"]
df['TransactionFee'] = 0
df['Status'] = "Successful"
df['Network'] = ''
df['TXID'] = ''

In [ ]:
df_borrow = df[df.type=="borrowIn"].copy()
df_borrow['SourceAddress'] = 'Binance Loans'
df_borrow['Address'] = ''
df_borrow = df_borrow[['Date(UTC)', 'Coin', 'Network', 'Amount', 'TransactionFee', 'Address', 'TXID', 'SourceAddress', 'PaymentID', 'Status']]

In [ ]:
df_borrow

In [ ]:
df_borrow.to_csv(r"C:\Users\lukem\Downloads\deposit_history.csv", index=False)

In [ ]:
df_repay = df[df.type=="repayAmount"].copy()
df_repay['SourceAddress'] = ''
df['Address'] = 'Binance Loans'
df_repay.Amount = df_repay.Amount * -1
df_repay = df_repay[['Date(UTC)', 'Coin', 'Network', 'Amount', 'TransactionFee', 'Address', 'TXID', 'SourceAddress', 'PaymentID', 'Status']]

In [ ]:
df_repay

In [ ]:
df_repay.to_csv(r"C:\Users\lukem\Downloads\withdrawal_history.csv", index=False)